In [ ]:
import requests
import websocket
import json
import logging
from time import sleep

## Request to create participation and get websocket url

In [ ]:
r = requests.post('http://localhost:8080/pre-chat/6287f9aa-a6b6-456f-8957-4e280007a47f', 
                  data={'firstName': 'John'})

In [ ]:
r.raise_for_status()

In [ ]:
chat_info = r.json()
chat_info

In [ ]:
chat_info['websocketUrl']

In [ ]:
def on_message(ws, message):
    # print(message)
    parsed = json.loads(message)
    if "data" in parsed:
        if "chat" in parsed["data"]:
            chat_data = parsed["data"]["chat"]
            if "role" in chat_data and chat_data["role"] == "rep":
                if chat_data["type"] == "normal":
                    print(chat_data["text"])
                elif chat_data["type"] == "user-joined":
                    print("=== Welcome to Juji Bot ===")

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")

def on_open(ws):
    print("Opened connection")

## Initiate Websocket App

In [ ]:
websocket.enableTrace(True)
ws = websocket.WebSocketApp(chat_info['websocketUrl'],
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

In [ ]:
ws.run_forever()

## Initiate Websocket

In [ ]:
ws = websocket.create_connection(chat_info['websocketUrl'])

In [ ]:
ws.send("""
        subscription {{
                chat(input: {{
                    participationId: "{0}"
                }}) {{
                    role
                    text
                    type
                    display{{
                        data{{
                            questions{{
                                heading
                                kind
                            }}
                        }}
                    }}
                }}
            }}""".format(chat_info["participationId"]))

In [ ]:
while True:
    sleep(1)
    result = ws.recv()
    on_message(ws, result)